# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
csv_path = os.path.join("..", "WeatherPy", "Output", "output.csv")
original_weather_df = pd.read_csv(csv_path)
original_weather_df[["Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Date"]] = original_weather_df[["Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Date"]].astype(float)
original_weather_df.dropna(axis=0,how='all', inplace=True)#drop all NaN items from data frame
original_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Puerto Ayora,-0.74,-0.74,72.00,90.0,27.0,3.00,EC,1.608036e+09
1,Qandala,11.47,11.47,77.04,58.0,10.0,10.71,SO,1.608036e+09
2,Albany,42.60,42.60,28.00,49.0,0.0,1.99,US,1.608035e+09
3,Barrow,71.29,71.29,-2.20,70.0,1.0,16.11,US,1.608035e+09
4,Busselton,-33.65,-33.65,63.00,63.0,0.0,5.01,AU,1.608036e+09
...,...,...,...,...,...,...,...,...,...
627,Lata,40.16,40.16,61.00,74.0,29.0,5.01,PT,1.608036e+09
628,Port-Cartier,50.03,50.03,16.00,66.0,100.0,10.29,CA,1.608036e+09
629,Pouébo,-20.40,-20.40,76.46,77.0,2.0,13.87,NC,1.608036e+09
630,Yuzhno-Sukhokumsk,44.66,44.66,46.27,75.0,100.0,16.37,RU,1.608036e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
#configure gmaps using my Google API key
gmaps.configure(api_key=g_key)

coordinates = original_weather_df[['Lat', 'Lng']].astype(float)
humidity = original_weather_df["Humidity"].astype(float)
# Customize the size of the figure
figure_layout = {
    'width': '533px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=0.20)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [20]:
good_weather_df = original_weather_df.copy()

good_weather_df = good_weather_df[(good_weather_df['Max Temp'] > 70) & (good_weather_df['Max Temp'] < 80)] #only show cities with a max temperature between 70 and 80 degrees
good_weather_df = good_weather_df[good_weather_df['Wind Speed'] > 10] #and only show cities where the wind speed is <10 mph
good_weather_df = good_weather_df[good_weather_df['Cloudiness'] == 0] #and only show cities where there's no clouds
good_weather_df.dropna(axis=0,how='any', inplace=True) #removing any of the cities that have null values

#good_weather_df
original_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Puerto Ayora,-0.74,-0.74,72.00,90.0,27.0,3.00,EC,1.608036e+09
1,Qandala,11.47,11.47,77.04,58.0,10.0,10.71,SO,1.608036e+09
2,Albany,42.60,42.60,28.00,49.0,0.0,1.99,US,1.608035e+09
3,Barrow,71.29,71.29,-2.20,70.0,1.0,16.11,US,1.608035e+09
4,Busselton,-33.65,-33.65,63.00,63.0,0.0,5.01,AU,1.608036e+09
...,...,...,...,...,...,...,...,...,...
627,Lata,40.16,40.16,61.00,74.0,29.0,5.01,PT,1.608036e+09
628,Port-Cartier,50.03,50.03,16.00,66.0,100.0,10.29,CA,1.608036e+09
629,Pouébo,-20.40,-20.40,76.46,77.0,2.0,13.87,NC,1.608036e+09
630,Yuzhno-Sukhokumsk,44.66,44.66,46.27,75.0,100.0,16.37,RU,1.608036e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df  = good_weather_df.copy()
hotel_df["Hotel Name"] = "NaN" #initialize Hotel Name column
hotel_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# find the closest hotel to coordinates. use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    cityName = row['City'] #get city name type from df
    lat = row['Lat'] #get latitude from df
    lng = row['Lng'] #get longitude from df
    
    print(lat)
    print(lng)
    
    params = {
    "location": f"{lat},{lng}", #substituting latitude and longitude into the API call
    "rankby": "distance",
    "type": "restaurant",
    "radius": 5000, #within 5000 meters of this location
    "key": g_key,
    }    

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {cityName}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    print(results)
    
#     try:
#         print(f"Closest {cityName} restaurant is {results[0]['name']}.")
        
# #         types_df.loc[index, 'name'] = results[0]['name']
# #         types_df.loc[index, 'address'] = results[0]['vicinity']
# #         types_df.loc[index, 'price_level'] = results[0]['price_level']
# #         types_df.loc[index, 'rating'] = results[0]['rating']
        
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")
        
#     print("------------")

-33.93
-33.93
Retrieving Results for Index 55: Cape Town.
[]
-20.23
-20.23
Retrieving Results for Index 58: Poum.
[]
-33.92
-33.92
Retrieving Results for Index 79: Port Elizabeth.
[]
-24.87
-24.87
Retrieving Results for Index 88: Carnarvon.
[]
-33.01
-33.01
Retrieving Results for Index 133: Saldanha.
[]
22.57
22.57
Retrieving Results for Index 154: Sur.
[]
20.93
20.93
Retrieving Results for Index 169: Nouadhibou.
[]
9.35
9.35
Retrieving Results for Index 232: Jijiga.
[]
14.69
14.69
Retrieving Results for Index 287: Dakar.
[]
25.26
25.26
Retrieving Results for Index 288: Pasni.
[]
9.49
9.49
Retrieving Results for Index 383: Bandarbeyla.
[]
20.9
20.9
Retrieving Results for Index 507: Verāval.
[]


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure

# fig = gmaps.figure(layout=figure_layout)  # Assign the marker layer to a variable
# markers = gmaps.marker_layer(coordinates)
# # Add the layer to the map
# fig.add_layer(markers)

# heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
#                                  dissipating=False, max_intensity=100,
#                                  point_radius = 0.05)
# fig.add_layer(heat_layer)
# fig
